In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

import pandas as pd
import sqlite3
import hashlib
import time
from sqlalchemy import text, inspect, MetaData, Table
from sqlalchemy.orm import sessionmaker
from app.main import getTables, Base, Account, Transaction, TransactionFinal, Load
from app.helpers.readCsv import openOneCsv, getAccountHistory
from app.helpers.sqlEngine import engine

pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 1000)


In [2]:
Session = sessionmaker(bind=engine)
session = Session()
session.connection().execute(text("DELETE FROM loads"))
session.connection().execute(text("DELETE FROM transactions_all"))
session.connection().commit()
session.close()

accounts = getTables(engine)

for index, account in accounts.iterrows():
    path = account['csvPath']
    print(path)
    
    Session = sessionmaker(bind=engine)
    session = Session()

    allFiles = getAccountHistory(path, account['id'], session)
    display(allFiles)
    allFiles.to_sql('transactions_all', con=session.connection(), if_exists='append', index=False, chunksize=10)
    session.connection().commit()
    session.close()


In [3]:
pd.read_sql(sql=text("""
    SELECT
        sha256,
        postDate,
        transactionDate,
        accountId,
        amount,
        description,
        details,
        type,
        category,
        balance,
        checkOrSlipNumber,
        count(*)
    FROM 
        transactions_all
    WHERE
        postDate >= '2023-01-05'
    GROUP BY
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11
    ORDER BY
        postDate DESC
"""), con=engine.connect())